In [393]:
pip install transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [394]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd


from transformers import AutoTokenizer
from transformers import AutoConfig




In [395]:
#load data
data = pd.read_csv('Symptom2Disease.csv')

#seperate data into the text and label
symptoms = data['text'].values
diseases = data['label'].values

print(len(symptoms))
print(len(diseases))
    


1200
1200


In [396]:
#connect to GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [397]:
#tokenize data and pad them to equal sequences
print(symptoms[0])

tokenizer = AutoTokenizer.from_pretrained("monologg/biobert_v1.1_pubmed")
symptoms = [tokenizer(symptom, padding = True, truncation = True) for symptom in symptoms]

print(f"After tokenizing: {symptoms}")







I have been experiencing a skin rash on my arms, legs, and torso for the past few weeks. It is red, itchy, and covered in dry, scaly patches.
After tokenizing: [{'input_ids': [101, 146, 1138, 1151, 13992, 170, 2241, 187, 10733, 1113, 1139, 1739, 117, 2584, 117, 1105, 16374, 1111, 1103, 1763, 1374, 2277, 119, 1135, 1110, 1894, 117, 1122, 8992, 117, 1105, 2262, 1107, 3712, 117, 188, 7867, 1183, 14879, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, {'input_ids': [101, 1422, 2241, 1144, 1151, 185, 19032, 1158, 117, 2108, 1113, 1139, 4257, 117, 14228, 117, 1105, 23658, 119, 1188, 185, 19032, 1158, 1110, 1510, 4977, 1118, 170, 4968, 1137, 25648, 8710, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [398]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
diseases = label_encoder.fit_transform(diseases)

print(diseases[91])

17


In [399]:
#convert to pytorch tensors





In [400]:
pip install scikit-learn

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [401]:
#split data into train and test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(symptoms, diseases, test_size=0.2)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


AttributeError: 'list' object has no attribute 'shape'

training text: torch.Size([960, 25])
training labels: torch.Size([960])
testing text: torch.Size([240, 25])
testing labels: torch.Size([240])
tensor([ 12, 112, 160,   2, 110, 111,  32, 161, 162,   2,  17,  18, 163,  40,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0])
training text: torch.Size([30, 32, 25])
training labels: torch.Size([30, 32, 1])
testing text: torch.Size([7, 32, 25])
testing labels: torch.Size([7, 32, 1])


/var/folders/bd/126yvkv50d1cjxc0144dqz4c0000gn/T/ipykernel_1987/567756006.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
/var/folders/bd/126yvkv50d1cjxc0144dqz4c0000gn/T/ipykernel_1987/567756006.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)


In [ ]:
#create a simple fnn model that takes in the input size
class FNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=0)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.softmax(out)
        return out
    
#rnn model
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.linear = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=1)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        out = self.rnn(x)[0]
        out = self.linear2(out)
        out = self.relu(out)
        out = self.linear2(out)
        out = self.relu(out)
        out = self.linear2(out)
        out = self.relu(out)
        out = self.linear(out)
        out = self.softmax(out)
        return out
    
#LSTM model
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=0)
        
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        out = self.softmax(out)
        return out
    def predict(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        out = self.softmax(out)
        return out


    
    




#### 25 represents the input size, 1 represents the prediction size



In [ ]:
#define hyperparameters
learning_rate = 1e-6
epochs = 50

input_size = X_train.shape[2]
hidden_size = 10
output_size = 1

print(f"Input size: {input_size}")
print(f"out size: {output_size}")





Input size: 25
out size: 1
tensor([15])
tensor([ 12, 112, 160,   2, 110, 111,  32, 161, 162,   2,  17,  18, 163,  40,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0])


In [ ]:
model_num = 1
model = None
if(model_num == 0):
    model = FNN(input_size, hidden_size, output_size)

elif(model_num == 1):
    #Reshape for RNN
    model = RNN(input_size, hidden_size, output_size)
elif(model_num == 1):

    config = AutoConfig.from_pretrained("bert-base-uncased", num_labels=5)
     model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", config=config)


model.to(device)

RNN(
  (rnn): RNN(25, 10, batch_first=True)
  (linear2): Linear(in_features=10, out_features=10, bias=True)
  (linear): Linear(in_features=10, out_features=1, bias=True)
  (softmax): Softmax(dim=1)
  (relu): ReLU()
)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
model.train()
#train the model

#training accuracy
correct = 0
total = 0

for epoch in range(epochs):
    
    for batch_idx in range(len(X_train)):
        
        #get the data
        data = X_train[batch_idx].float().to(device)
        target = y_train[batch_idx].long().to(device)

        #add the extra dimension into data to make it 3d
        # data format: [batch_size, seq_len, input_size]
        data = data.unsqueeze(1).expand(-1, num_test_batches, -1)
        
        #forward pass
        print(data.shape)
        print(target.shape)
        output = model(data)
       
        output = output[:, -1, :]
        print(output.shape)
        print(len(output))
        print(target)
        loss = criterion(output, target.squeeze(1))
        
        #backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        #get the training accuracy
        _, predicted = torch.max(output, 1)
        total += target.size(0)
        print(predicted.shape)
        print(target.shape)

        correct += (predicted == target.squeeze(1)).sum().item()


    if(epoch % 10 == 0):
        print(f"Epoch: {epoch+1}/{epochs}, Loss: {loss.item():.4f}, Accuracy: {100*correct/total:.2f}%")

        
    





    

        
    


torch.Size([32, 7, 25])
torch.Size([32, 1])
torch.Size([32, 1])
32
tensor([[15],
        [21],
        [18],
        [ 0],
        [22],
        [ 2],
        [17],
        [14],
        [18],
        [19],
        [10],
        [ 2],
        [ 3],
        [ 3],
        [18],
        [14],
        [10],
        [ 4],
        [ 5],
        [23],
        [15],
        [ 3],
        [ 3],
        [12],
        [ 4],
        [ 4],
        [13],
        [ 9],
        [21],
        [14],
        [22],
        [ 5]])


IndexError: Target 15 is out of bounds.

In [ ]:
model.eval()
#test the model
correct = 0
total = 0
with torch.no_grad():
    for batch_idx in range(len(X_test)):
        
        #get the data
        data = X_test[batch_idx].to(device).float()
        target = y_test[batch_idx].to(device).long()
        
        #forward pass
        output = model(data)
        loss = criterion(output, target.squeeze())
        
        #get the training accuracy
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target.squeeze()).sum().item()
    print(f"Test Accuracy: {100*correct/total:.2f}%")

Test Accuracy: 3.57%


In [ ]:
#f1score
from sklearn.metrics import f1_score
y_true = []
y_pred = []
with torch.no_grad():
    for batch_idx in range(len(X_test)):
        data = X_test[batch_idx].float().to(device)
        target = y_test[batch_idx].long().to(device)
        
        output = model(data)
        _, predicted = torch.max(output.data, 1)
        y_true.extend(target.squeeze().tolist())
        y_pred.extend(predicted.tolist())

print(f"F1 score: {f1_score(y_true, y_pred, average='macro')}")
print(f"F1 score: {f1_score(y_true, y_pred, average='micro')}")
print(f"F1 score: {f1_score(y_true, y_pred, average='weighted')}")
print(f"F1 score: {f1_score(y_true, y_pred, average=None)}")




F1 score: 0.005624027761158311
F1 score: 0.03571428571428571
F1 score: 0.0068088770748217925
F1 score: [0.         0.         0.         0.         0.         0.03278689
 0.10218978 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.        ]
